<img src="https://www.colorado.edu/rc/sites/default/files/page/logo.png"
     alt="Logo for Research Computing @ University of Colorado Boulder"
     width="400" />

# Scatter/Gather of data

One common task when parallel programming involves distributing (scattering) a list of numbers among
the different processes or collating (gathering) a distributed list of numbers back to the hub processes.
This example illustrates the basic mechanics of scattering
 and gathering.
 
 <img src="scatter.png"
     alt="Concept of scatter a list to engines "
     width="215" />
<img src="gather.png"
     alt="Concept of gather the values a list to engines "
     width="200" />

In [ ]:
!ipcluster start -n 4 --daemonize

In [ ]:
import ipyparallel

profile = 'example-shas'
rc = ipyparallel.Client(profile=profile)
nengines = len(rc)
nengines

## Create list of data

In [ ]:
all_proc  = rc[:]
all_proc.block=True

a = []
lsize=6*nengines
for i in range(0,lsize):
    a.append(i**2)

a

## Scatter the list

We scatter the list "a" from the hub out to all engines. 
Each process stores a portion of "a" locally in the variable "mylist"

In [ ]:
all_proc.scatter('mylist',a)

In [ ]:
%%px
print(mylist)

## Get `mylist` to list of lists

Create a variable on the controller that holds the contents of `mylist` for each engine.
sub_lists is a nested list, `sub_list[i][:]` holds the value `mylist`for engine 'i'

In [ ]:
sub_lists = all_proc['mylist']

print('\n ',nengines," Python engines are active.\n")

print(' ')
for i in range(nengines):
    istr = '{:02d}'.format(i)  # returns a 2-digit string whose value is i
    msg = 'Engine '+istr+':   list segment = '
    print(msg, sub_lists[i])
print(' ')

## Gather the `mylist` data

Gather `mylist` back to the controller, store the contents in a list named gathered.

In [ ]:
gathered = all_proc.gather('mylist')
print('Gathered list: ', gathered[:], type(gathered))

In [ ]:
print(sub_lists)

In [ ]:
!ipcluster stop